In [11]:
import os
from langchain.agents import load_tools, AgentType
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain import pipeline

In [3]:
pipe = pipeline("visual-question-answering", model="dandelin/vilt-b32-finetuned-vqa")

config.json:   0%|          | 0.00/136k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [35]:
img_text = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

In [32]:
openai_api_key = 'sk-*****************************************************'
os.environ["SERPAPI_API_KEY"] = "8ac***********************************************"

llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)


tools = load_tools(['serpapi'], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [67]:
def VQA_with_LLM(image, question):

    vqa_answer  = pipe({"image":image, "question":question})[0]['answer']
    img_caption  = img_text(image)[0]['generated_text']

    template = """
    Given the question: '{question}',
    the image caption: '{image_caption}',
    and the VQA model's output: '{vqa_answer}',
    By considering these information, can you answer the question more comprehensively or provide additional insights?"
    Also you should not mention any clues or references like the vqa_output or answers that you have considered in the response.

    """

    prompt = PromptTemplate(
        input_variables=["question", "image_caption", "vqa_answer"],
        template=template,
    )

    final_prompt = prompt.format(question = question, image_caption = img_caption, vqa_answer = vqa_answer)
    agent.run(final_prompt)



> Entering new AgentExecutor chain...
 I need to look at the information provided and determine how to answer the question.
Action: Search 
Action Input: "Are dogs the best pet in the world?" 
Observation: There are many reasons why dogs are known as man's best friend. They entertain us with their antics, greet us with an exuberant wag of the tail every day (no matter how many times you leave and come back), and offer endless photo ops to fill our social feeds.
Thought: I need to look at the other information provided.
Action: Search 
Action Input: "Do dogs make good pets?"
Observation: Dogs are brilliant companions One of the most noticeable benefits of owning a dog is that it's almost impossible to feel lonely when your dog is by your side, and for good reason. Most dogs are highly sociable, and they love company, whether it's canine or human.
Thought: I now know the final answer.
Final Answer: Dogs make excellent pets and have been known as man's best friend for many years. They p

"Dogs make excellent pets and have been known as man's best friend for many years. They provide companionship, entertainment, and unconditional love. Although in the image provided the dogs are not running, they can still make excellent pets."

In [ ]:
image = "/content/sample_data/dogs.jpeg"
question = "Are these dogs running? are dogs are the best pet in the world?"

if __name__ == '__main__':
    VQA_with_LLM(image, question)